In [ ]:
#default_exp pleiades

# Pleiades Example

> Working through conversion to cogs, upload to object storage, stack item metadata parsing, etc. for pleiades datasets

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from sac_stac.utils import sedas_find_datasets, sedas_download, sedas_extract

In [ ]:
import pandas as pd
import os

In [ ]:
import geopandas as gpd

## Find some useful samples

Probably best if we find some samples to work through the process.

In [ ]:
result = sedas_find_datasets("POLYGON((-1.91 51.81,-1.15 51.81,-1.15 51.50,-1.91 51.50,-1.91 51.81))", 
                             "2000-01-01T00:00:00Z", 
                             "2020-10-27T00:00:00Z",
                             "Pleiades"
                            )
pd.DataFrame(result['products'])

,productId,supplierId,type,satelliteName,instrumentName,modeName,sensorType,sensorResolution,coordinatesWKT,start,...,area,aoiCoveragePercent,usefulAreaPercent,cloudCoveragePercent,productType,latency,ql,thumbnail,vendorSpecific,downloadUrl
0,c0ec9a5f12356e87bc910ddbc49dbb76,Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.654728 51.309517,-1.345414 51.3081...",2018-10-24T11:17:22Z,...,4.825125e+08,1.0,5.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'cf...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
1,94cc7887414be0912de7ca44288f79da,Pleiades_UKSA174_SO18034614-74-01_DS_PHR1A_201...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-2.153852 51.603313,-1.841533 51.6033...",2018-09-29T11:10:08Z,...,1.565260e+08,2.0,22.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '4b...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
2,6ac7da92a92161839c0bf29ab1b878d3,Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.903588 51.628784,-1.592515 51.6281...",2018-09-29T11:10:00Z,...,1.796598e+08,10.0,100.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'e6...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
3,dd08a178209ed51cc57f5caef1e1ad7c,Pleiades_UKSA341_SO18034616-41-01_DS_PHR1B_201...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.401923 51.155725,-1.101068 51.1538...",2018-09-02T11:17:34Z,...,8.211908e+08,1.0,1.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'a5...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
4,97528406bc14d1c94ef22d20b4b26c7a,Pleiades_UKSA305_SO18034616-5-01_DS_PHR1B_2018...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.576165 51.728406,-1.265721 51.7268...",2018-07-24T11:24:30Z,...,4.256422e+08,11.0,46.0,1.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '77...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
5,3e8d4c9096b3ba5c7126499e442424f5,Pleiades_UKSA87_SO18034613-87-01_DS_PHR1A_2018...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.413484 51.469752,-1.089638 51.4676...",2018-06-29T11:18:17Z,...,9.264473e+08,34.0,67.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '7f...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
6,b5ff24c70eba28f3ec7292c356fb27b0,Pleiades_UKSA204_SO18034615-4-01_DS_PHR1B_2018...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.928964 51.516819,-1.63821 51.51628...",2018-02-25T11:21:25Z,...,2.385591e+08,12.0,94.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '4b...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
7,92fe8121944a5267861d31597afb016a,Pleiades_UKSA7_SO18034613-7-01_DS_PHR1A_201802...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.606218 51.306711,-1.294645 51.3052...",2018-02-24T11:27:55Z,...,6.639221e+08,11.0,29.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'ef...",https://sedasdm.satapps.org/datamgr/datamgr.ph...


## Use single example

In [ ]:
wd = "/tmp/data/"

scene_res = result['products'][0]
scene_name = sedas_res['supplierId']
scene_name

'Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712'

In [ ]:
scene_res

{'productId': 'c0ec9a5f12356e87bc910ddbc49dbb76',
 'supplierId': 'Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712',
 'type': 'ARCHIVE',
 'satelliteName': 'Pleiades-1B',
 'instrumentName': 'MS/PAN',
 'modeName': '0.000',
 'sensorType': 'Optical',
 'sensorResolution': 2.0,
 'coordinatesWKT': 'POLYGON((-1.654728 51.309517,-1.345414 51.308197,-1.342538 51.509259,-1.653212 51.510588,-1.654728 51.309517))',
 'start': '2018-10-24T11:17:22Z',
 'stop': '2018-10-24T11:17:24Z',
 'orbitDirection': 'DESCENDING',
 'area': 482512528.74,
 'aoiCoveragePercent': 1.0,
 'usefulAreaPercent': 5.0,
 'cloudCoveragePercent': 0.0,
 'productType': 'L3',
 'latency': 'Standard',
 'ql': 'https://geobrowser.satapps.org/archiveql/aewebpub/getaeql.php?aeimgid=c0ec9a5f12356e87bc910ddbc49dbb76&aeimgtype=ql&epsg=3857&',
 'thumbnail': 'https://sedasdm.satapps.org/qls/qlmgr.php?sceneid=UKSA/pleiades/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712/c0e

## Download & Extract

create zip path to download into from sedas result

In [ ]:
down_zip = f"{wd}{scene_res['supplierId']}.zip"
down_zip

'/tmp/data/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712.zip'

download the scene

In [ ]:
sedas_download(scene_res, down_zip)

In [ ]:
os.path.exists(down_zip)

True

create a scene directory path to extract into

In [ ]:
scene_dir = f"{down_zip[:-4]}/"
scene_dir

'/tmp/data/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712/'

extract

In [ ]:
sedas_extract(down_zip, scene_dir)

## Mosaic & COG

## Export

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
